In [1]:
import pickle
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
import gzip
import shutil
# import os
import pandas as pd
from sklearn.utils import shuffle

from joblib import Parallel, delayed
import tqdm
import pyarrow
# Paths of datasets



ranked_full_game_save_path = 'C:/Users/jaspa/Grant ML/ranked_full_subfolders'
ranked_segment_save_path_subfolder = 'C:/Users/jaspa/Grant ML/ranked_segments_subfolders'
ranked_segment_save_path_bulk = 'C:/Users/jaspa/Grant ML/ranked_segments_bulk'

public_full_game_save_path = 'C:/Users/jaspa/Grant ML/public_full_subfolders'
public_segment_save_path_subfolder = 'C:/Users/jaspa/Grant ML/public_segments_subfolders'
public_segment_save_path_bulk = 'C:/Users/jaspa/Grant ML/public_segments_bulk'

List all slp game input arrays we want to process. Make a dataframe with the data.

In [2]:
# def list_files(directory):
#     for root, dirs, files in os.walk(directory):
#         for file in files:
#             yield os.path.join(root, file)

# def parse_file_path(file_path):
#     parts = file_path.split('\\')
#     # Extracting necessary details from the file path
#     file_name = parts[-1]
#     character = parts[-2]
#     opponent = parts[-3]
#     return {
#         'file': file_name,
#         'path': '\\'.join(parts[:-1]),
#         'character': character,
#         'opponent': opponent,
#         'game_length': 0,  # Placeholder values
#         'segment_shift': 0,
#         'num_segments': 0
#     }

# def generate_dataframe_from_folders(folders):
#     files_data = []
#     for folder in folders:
#         for file_path in list_files(folder):
#             files_data.append(parse_file_path(file_path))
#     return pd.DataFrame(files_data)

# ranked_full_game_save_path = 'C:/Users/jaspa/Grant ML/ranked_full_subfolders'
# public_full_game_save_path = 'C:/Users/jaspa/Grant ML/public_full_subfolders'
# folders = [ranked_full_game_save_path, public_full_game_save_path]

# df = generate_dataframe_from_folders(folders)
# print(df.head())  # Display the first few rows to check

Use Joblib to open the game arrays and get the length of the game in frames. Save the data frame. About 40 seconds.

In [3]:
# def get_array_length(file_path):
#     # Update this function based on how you want to handle the file reading,
#     # For gzipped numpy files, you would do something like this:
#     with gzip.open(file_path, 'rb') as f:
#         arr = np.load(f)
#         return arr.shape[1]

# def process_row(row):
#     # Call the get_array_length function with the path of the file
#     length = get_array_length(row['path'] + '\\' + row['file'])
#     return length


# num_cores = -1  # Or however many cores you want to use
# lengths = Parallel(n_jobs=num_cores,verbose = 0)(delayed(process_row)(row) for index, row in tqdm.tqdm(df.iterrows()))
# df['game_length'] = lengths

# df.to_feather('C:/Users/jaspa/Grant ML/slp/data/path_info_df.feather')


# print(df.head())  # To check the updated DataFrame


In [4]:
# Define the path to your feather file
file_path = 'C:/Users/jaspa/Grant ML/slp/data/path_info_df.feather'

# Load the feather file
df = pd.read_feather(file_path)

Set the number of segments you want to use (same for each character). Check what the maximum power of two you need to shift each segment by to get the right number of segments. Can shift by as little as 64 frames.

In [5]:
min_segments = 20000
segment_length = 1024
for character in df['character'].unique():
    game_lengths = df.loc[df['character'] == character, 'game_length']
    for i in range(5):
        segment_shift = 2 ** (10-i)
        num_segments = 0
        for game_length in game_lengths:
            num_segments += (game_length - segment_length) // segment_shift
        if num_segments > min_segments:
            break
    # Correct way to set values to avoid SettingWithCopyWarning
    df.loc[df['character'] == character, 'segment_shift'] = segment_shift

print(df.head())

                                       file  \
0  107729__1__CAPTAIN_FALCON__BOWSER.npy.gz   
1    1122__0__BOWSER__CAPTAIN_FALCON.npy.gz   
2  113580__0__BOWSER__CAPTAIN_FALCON.npy.gz   
3  116020__0__BOWSER__CAPTAIN_FALCON.npy.gz   
4   13522__0__BOWSER__CAPTAIN_FALCON.npy.gz   

                                                path       character opponent  \
0  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   
1  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   
2  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   
3  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   
4  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   

   game_length  segment_shift  num_segments  
0        10935           1024             0  
1        11244           1024             0  
2         7487           1024             0  
3        12676           1024             0  


Check how many segments you will get from each game.

In [6]:
df['num_segments'] = (df['game_length']- segment_length) // df['segment_shift']
print(df.head())

                                       file  \
0  107729__1__CAPTAIN_FALCON__BOWSER.npy.gz   
1    1122__0__BOWSER__CAPTAIN_FALCON.npy.gz   
2  113580__0__BOWSER__CAPTAIN_FALCON.npy.gz   
3  116020__0__BOWSER__CAPTAIN_FALCON.npy.gz   
4   13522__0__BOWSER__CAPTAIN_FALCON.npy.gz   

                                                path       character opponent  \
0  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   
1  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   
2  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   
3  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   
4  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   BOWSER   

   game_length  segment_shift  num_segments  
0        10935           1024             9  
1        11244           1024             9  
2         7487           1024             6  
3        12676           1024            11  


See haw many extra segments you have for each character.

In [7]:
# min_segments = 20000
# segment_length = 1024
for character in df['character'].unique():
    character_df = df.loc[df['character'] == character]
    print(character, ': ', sum(character_df['num_segments'])-min_segments)


CAPTAIN_FALCON :  297909
DONKEY_KONG :  434
DR_MARIO :  19104
FALCO :  621333
FOX :  816085
GAME_AND_WATCH :  15886
GANONDORF :  42149
ICE_CLIMBERS :  57542
JIGGLYPUFF :  148477
KIRBY :  6073
LINK :  6538
LUIGI :  36019
MARIO :  16559
MARTH :  424374
MEWTWO :  1621
NESS :  8442
PEACH :  173956
PICHU :  623
PIKACHU :  23279
ROY :  5573
SAMUS :  93468
SHEIK :  262434
YOSHI :  32627
YOUNG_LINK :  11191
ZELDA :  8814
BOWSER :  961


Randomize the games from each character. Cound how many segments you are going to get for each character. Split the games up so that 15% of the segments end up in the test set and the validation and 70% of the segments end up in the train set. No segments from the same game will end up in different data sets.

In [8]:


# Initialize empty DataFrames for test, validation, and training sets
test_df = pd.DataFrame()
val_df = pd.DataFrame()
train_df = pd.DataFrame()

min_segments = 20000
# target_segments_for_test_and_val = min_segments * 0.15

for character in df['character'].unique():
    character_df = df.loc[df['character'] == character].sample(frac=1).reset_index(drop=True)  # Shuffle the rows
    sum_segments = 0
    test_idx = 0
    val_idx = 0
    
    #split the games of each character by .15, .15, and .7
    target_segments_for_test_and_val = sum(character_df['num_segments']) * 0.15
    # Determine the split indices for test and validation sets
    for i, row in character_df.iterrows():
        sum_segments += row['num_segments']
        if sum_segments >= target_segments_for_test_and_val:
            if test_idx == 0:  # First batch for test set
                test_idx = i
                sum_segments = 0  # Reset sum for validation set calculation
            elif val_idx == 0:  # Next batch for validation set
                val_idx = i
                break
    
    # Split the character_df into test, val, and train based on the indices
    test_rows = character_df.iloc[:test_idx+1]
    val_rows = character_df.iloc[test_idx+1:val_idx+1]
    train_rows = character_df.iloc[val_idx+1:]

    # Append to respective DataFrames
    test_df = pd.concat([test_df, test_rows], ignore_index=True)
    val_df = pd.concat([val_df, val_rows], ignore_index=True)
    train_df = pd.concat([train_df, train_rows], ignore_index=True)
    
print(test_df.shape)
print(val_df.shape)
print(train_df.shape)

print(train_df.head())

(62857, 7)
(62831, 7)
(293391, 7)
                                               file  \
0           250642__0__MARTH__CAPTAIN_FALCON.npy.gz   
1  233849__1__CAPTAIN_FALCON__CAPTAIN_FALCON.npy.gz   
2       108447__0__GANONDORF__CAPTAIN_FALCON.npy.gz   
3       53622__1__CAPTAIN_FALCON__JIGGLYPUFF.npy.gz   
4               4174__0__FOX__CAPTAIN_FALCON.npy.gz   

                                                path       character  \
0  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON   
1  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON   
2  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   
3  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   
4  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   

         opponent  game_length  segment_shift  num_segments  
0           MARTH        10871           1024             9  
1  CAPTAIN_FALCON         9740           1024             8  
2       GANONDOR

Expand the data frames so that there is one path for each segment.

In [9]:
# import pandas as pd
# import numpy as np

def expand_df_vectorized(df):
    # Calculate the repeat count for each row based on 'num_segments'
    repeats = df['num_segments'].values
    
    # Repeat each index according to its corresponding 'num_segments' value
    index_repeated = np.repeat(df.index, repeats)
    
    # Create a new DataFrame by repeating rows
    df_repeated = df.loc[index_repeated].reset_index(drop=True)
    
    # Create a 'segment_index' column that counts up for each group of repeated rows
    segment_indices = np.concatenate([np.arange(n) for n in repeats])
    
    # Assign 'segment_index' to the repeated DataFrame
    df_repeated['segment_index'] = segment_indices
    
    # Optionally, drop the 'num_segments' column if it's no longer needed
    df_repeated = df_repeated.drop(columns=['num_segments'])
    
    return df_repeated


# # Example usage:
expanded_test_df = expand_df_vectorized(test_df)
expanded_val_df = expand_df_vectorized(val_df)
expanded_train_df = expand_df_vectorized(train_df)

print(expanded_test_df.shape)
print(expanded_val_df.shape)
print(expanded_train_df.shape)
print(expanded_test_df.head())

(547961, 7)
(547923, 7)
(2555587, 7)
                                      file  \
0  242318__3__CAPTAIN_FALCON__ZELDA.npy.gz   
1  242318__3__CAPTAIN_FALCON__ZELDA.npy.gz   
2  242318__3__CAPTAIN_FALCON__ZELDA.npy.gz   
3  242318__3__CAPTAIN_FALCON__ZELDA.npy.gz   
4  242318__3__CAPTAIN_FALCON__ZELDA.npy.gz   

                                                path       character opponent  \
0  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON    ZELDA   
1  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON    ZELDA   
2  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON    ZELDA   
3  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON    ZELDA   
4  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON    ZELDA   

   game_length  segment_shift  segment_index  
0         7450           1024              0  
1         7450           1024              1  
2         7450           1024              2  
3         7

Randomly sample the right number of segments to create each data set.  Encode the character.

In [10]:
def sample_rows_per_character(df, proportion, min_segments, encoder):
    rows_per_character = int(min_segments * proportion)
    sampled_df = pd.DataFrame()
    
    for character in df['character'].unique():
        character_df = df[df['character'] == character]
        sampled_rows = character_df.sample(n=min(rows_per_character, len(character_df)), random_state=1)
        sampled_df = pd.concat([sampled_df, sampled_rows], ignore_index=True)
    
    # Add the 'labels' column using the fitted encoder
    sampled_df['labels'] = encoder.transform(sampled_df['character'])
    
    return sampled_df


# Assuming 'characters' is your array of unique character names
characters = ['CAPTAIN_FALCON', 'DONKEY_KONG', 'DR_MARIO', 'FALCO', 'FOX',
              'GAME_AND_WATCH', 'GANONDORF', 'ICE_CLIMBERS', 'JIGGLYPUFF',
              'KIRBY', 'LINK', 'LUIGI', 'MARIO', 'MARTH', 'MEWTWO', 'NESS',
              'PEACH', 'PICHU', 'PIKACHU', 'ROY', 'SAMUS', 'SHEIK', 'YOSHI',
              'YOUNG_LINK', 'ZELDA', 'BOWSER']

# Initialize and fit the LabelEncoder
encoder = LabelEncoder()
encoder.fit(characters)

sampled_test_df = sample_rows_per_character(expanded_test_df, 0.15, min_segments, encoder)
sampled_val_df = sample_rows_per_character(expanded_val_df, 0.15, min_segments, encoder)
sampled_train_df = sample_rows_per_character(expanded_train_df, 0.70, min_segments, encoder)


print(sampled_test_df.shape)
print(sampled_val_df.shape)
print(sampled_train_df.shape)

print(sampled_test_df.head())

(78000, 8)
(78000, 8)
(364000, 8)
                                               file  \
0             264047__3__CAPTAIN_FALCON__FOX.npy.gz   
1            24132__1__CAPTAIN_FALCON__MARTH.npy.gz   
2  247147__1__CAPTAIN_FALCON__CAPTAIN_FALCON.npy.gz   
3            59959__0__FALCO__CAPTAIN_FALCON.npy.gz   
4            31335__1__CAPTAIN_FALCON__MARTH.npy.gz   

                                                path       character  \
0  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON   
1  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   
2  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON   
3  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   
4  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON   

         opponent  game_length  segment_shift  segment_index  labels  
0             FOX        10708           1024              0       1  
1           MARTH         8775           1024              5

In [11]:
sampled_test_df.to_feather('C:/Users/jaspa/Grant ML/slp/data/sample_test_df.feather')
sampled_val_df.to_feather('C:/Users/jaspa/Grant ML/slp/data/sample_val_df.feather')
sampled_train_df.to_feather('C:/Users/jaspa/Grant ML/slp/data/sample_train_df.feather')
